
# 🚗 Dynamic Pricing for Urban Parking Lots
**Capstone Project – Summer Analytics 2025**  
Libraries: `numpy`, `pandas`, `pathway`, `bokeh`  
Models: Baseline (Model 1), Demand-based (Model 2), Competitive (Model 3)  


In [1]:

!pip install pathway bokeh


In [2]:

import pandas as pd
import numpy as np

df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(['SystemCodeNumber', 'Timestamp'])
df['TrafficValue'] = df['TrafficConditionNearby'].map({'low': 1, 'medium': 2, 'high': 3})
df['VehicleWeight'] = df['VehicleType'].map({'car': 1.0, 'bike': 0.5, 'truck': 1.5})
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,TrafficValue,VehicleWeight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1.0,1.0
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,1.0,1.0
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,1.0,1.0
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,1.0,1.0
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,1.0,0.5


In [3]:

def model1_baseline(df, alpha=1.0, base_price=10.0):
    df = df.copy()
    df['Price_Model1'] = np.nan
    prices = {}
    for idx, row in df.iterrows():
        lot = row['SystemCodeNumber']
        occupancy = row['Occupancy']
        capacity = row['Capacity']
        price = base_price if lot not in prices else prices[lot] + alpha * (occupancy / capacity)
        df.at[idx, 'Price_Model1'] = price
        prices[lot] = price
        # print(df.head(2))
    return df


In [4]:

def model2_demand(df, base_price=10.0, λ=1.0):
    α, β, γ, δ, ε = 1.0, 0.7, 0.5, 1.0, 0.8
    df['DemandRaw'] = (
        α * (df['Occupancy'] / df['Capacity']) +
        β * df['QueueLength'] -
        γ * df['TrafficValue'] +
        δ * df['IsSpecialDay'] +
        ε * df['VehicleWeight']
    )
    df['DemandNorm'] = (df['DemandRaw'] - df['DemandRaw'].min()) / (df['DemandRaw'].max() - df['DemandRaw'].min())
    df['Price_Model2'] = base_price * (1 + λ * df['DemandNorm'])
    df['Price_Model2'] = df['Price_Model2'].clip(lower=0.5 * base_price, upper=2.0 * base_price)
    # print(df.head())
    return df


In [5]:

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def model3_competitive(df):
    df = df.copy()
    df['Price_Model3'] = df['Price_Model2']
    df['SuggestReroute'] = False
    location_lookup = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first().to_dict('index')
    proximity_map = {}
    for lot1, loc1 in location_lookup.items():
        nearby = []
        for lot2, loc2 in location_lookup.items():
            if lot1 != lot2:
                dist = haversine(loc1['Latitude'], loc1['Longitude'], loc2['Latitude'], loc2['Longitude'])
                if dist <= 0.5:
                    nearby.append(lot2)
        proximity_map[lot1] = nearby

    for idx, row in df.iterrows():
        lot = row['SystemCodeNumber']
        time = row['Timestamp']
        price = row['Price_Model2']
        capacity = row['Capacity']
        occupancy = row['Occupancy']
        competitors = proximity_map.get(lot, [])
        comp_prices = df[
            (df['SystemCodeNumber'].isin(competitors)) &
            (df['Timestamp'] == time)
        ]['Price_Model2'].values

        if len(comp_prices) > 0:
            avg_price = np.mean(comp_prices)
            if occupancy >= capacity and price > avg_price:
                df.at[idx, 'Price_Model3'] = max(avg_price * 0.95, 5)
                df.at[idx, 'SuggestReroute'] = True
            elif price < avg_price:
                df.at[idx, 'Price_Model3'] = min(price * 1.10, 20)
    # print(df.head())
    return df


In [6]:
print("model 1")
df = model1_baseline(df)
print("model 2")
df = model2_demand(df)
print("model 3")
df = model3_competitive(df)


Streaming output truncated to the last 5000 lines.
   Price_Model1  
0     10.000000  
1     10.110919  
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   

  LastUpdatedTime           Timestamp  TrafficValue  VehicleWeight  \
0        07:59:00 2016-10-04 07:59:00           1.0            1.0   
1        08:25:00 2016-10-04 08:25:00           1.0            1.0   

   Price_Model1  
0     10.000000  
1     10.110919  
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1     

In [7]:

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

def plot_lot_prices(lot_id):
    lot_df = df[df['SystemCodeNumber'] == lot_id]
    source = ColumnDataSource(lot_df)
    p = figure(title=f"Dynamic Pricing - {lot_id}", x_axis_type='datetime', width=800, height=350)
    p.line(x='Timestamp', y='Price_Model1', source=source, legend_label='Model 1', color='blue')
    p.line(x='Timestamp', y='Price_Model2', source=source, legend_label='Model 2', color='green')
    p.line(x='Timestamp', y='Price_Model3', source=source, legend_label='Model 3', color='red')
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    p.legend.location = 'top_left'
    show(p)

# Show plot for first lot
plot_lot_prices(df['SystemCodeNumber'].iloc[0])
